In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import timedelta, date
from time import sleep
import os

In [62]:
# 초기화

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(3)

driver.get('http://marketdata.krx.co.kr/contents/MKD/04/0406/04060100/MKD04060100.jsp')

def fc(selector):
    driver.find_element_by_css_selector(selector).click()
    sleep(1)
    
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(3) > button')
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(3) > ul > li:nth-child(4) > a')
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(4)')
fc('body > div.html-wrap > div.header-wrap > div > div.breadcrumb-wrap > div > div:nth-child(4) > ul > li:nth-child(2) > a')
fc('#market_gubun1a1d0c6e83f027327d8461063f4ac58a6') # 코스피만 받는다

In [63]:
def find_and_download(datetime_str):
    os.system('rm ~/Downloads/data.csv')
    
    date_form = driver.find_element_by_css_selector('#schdate6c8349cc7260ae62e3b1396831a8398f')
    date_form.send_keys(Keys.CONTROL, 'a')
    date_form.send_keys('\b')
    date_form.send_keys(datetime_str)

    fc('#btnidd645920e395fedad7bbbed0eca3fe2e0')
    sleep(5)

    fc('#d9d4f495e875a2e075a1a4a6e1b9770f > button:nth-child(7)')
    sleep(5)
    
    os.system(f'mv ~/Downloads/data.csv ./data/{datetime_str}.csv')

In [7]:
# 오늘까지 크롤링

d = date(2001, 1, 1)
t = date.today()

while d < t:
    d_str = d.strftime('%Y%m%d')
    find_and_download(d_str)
    d += timedelta(1)

In [64]:
# 누락된 파일 재시도

filelist = os.listdir('./data')
filelist = [x.split('.')[0] for x in filelist]

d = date(2001, 1, 1)
t = date.today()

while d < t:
    d_str = d.strftime('%Y%m%d')
    if d_str not in filelist:
        print(d_str)
        find_and_download(d_str)
    d += timedelta(1)


In [66]:
# 주말 제외한 휴일 재시도
d = date(2001, 1, 1)
t = date.today()

while d < t:
    d_str = d.strftime('%Y%m%d')
    filepath = './data/' + d_str + '.csv'

    # 주말이 아닌데 빈 파일인 경우
    if os.stat(filepath).st_size < 1024 and d.weekday() not in [5, 6]:
        find_and_download(d_str)
        
    d += timedelta(1)

In [68]:
# MD5 중복된 파일 검사 후 재시도

import hashlib

filelist = os.listdir('./data')
filelist.sort()

md5_dic = {}
for filename in filelist:
    filepath = './data/' + filename
    
    if os.stat(filepath).st_size < 1024:  # 1K 이하 파일은 거른다
        continue
    
    with open(filepath, 'rb') as f:
        k = hashlib.md5(f.read()).hexdigest()
    
    if k in md5_dic:
        md5_dic[k].append(filepath)
    else:
        md5_dic[k] = [filepath]

same_files = []
for k in md5_dic:
    if len(md5_dic[k]) > 1:
        print(md5_dic[k])
        same_files += md5_dic[k]
        
retry_dates = [x.split('/')[-1][:8] for x in same_files]

for date_str in retry_dates:
    find_and_download(date_str)